In [5]:
import numpy as np

In [6]:
def im2col(input_data: np.array, filter_h, filter_w, stride=1, pad=0):
    N, C, H, W = input_data.shape
    
    out_h = (H + 2 * pad - filter_h) // stride + 1
    out_w = (W + 2 * pad - filter_w) // stride + 1
    
    img = np.pad(input_data, [(0, 0), (0, 0), (pad, pad), (pad, pad)], 'constant')
    
    col = np.zeros((N, C, filter_h, filter_w, out_h, out_w))
    
    for y in range(filter_h):
        y_max = y + stride * out_h
        for x in range(filter_w):
            x_max = x + stride * out_w
            col[:, :, y, x, :, :] = img[:, :, y:y_max:stride, x:x_max:stride]
            
    col = col.transpose(0, 4, 5, 1, 2, 3).reshape(N * out_h * out_w, -1)
    
    return col
            


In [7]:
class Convolution(object):
    def __init__(self, W, b, stride=1, pad=0):
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad
        
        self.x = None
        self.col = None
        self.col_W = None
        
        self.dW = None
        self.db = None
        
    def forward(self, x):
        
        FN, C, FH, FW = self.W.shape
        N, C, H, W = x.shape
        
        out_h = 1 + int((H + 2 * self.pad - FH) / self.stride)
        out_w = 1 + int((W + 2 * self.pad - FW) / self.stride)
        
        col = im2col(x, FH, FW, self.stride, self.pad)
        
        print("input data -> im2col is ", col)
        
        col_W = self.W.reshape(FN, -1).T
        print("Weight = filter ... -> im2col is \n", col_W, end='\n')
        
        out = np.dot(col, col_W) + self.b
        print("affine 연상 수행 결과 \n", out, end='\n')
        
        out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2)
        
        self.x = x
        self.col = col
        self.col_W = col_W
        
        return out

In [9]:
filter_num = 1
input_channels = 1

x1 = np.array(
    [
        [0, 0, 0, 0, 0],
        [0, 1, 2, 3, 0],
        [0, 4, 5, 6, 0],
        [0, 7, 8, 9, 0],
        [0, 0, 0, 0, 0],
    ]
).reshape(1, input_channels, 5, 5)
print("input data is \n", x1, end='\n')

W1 = np.array([
    [0, 0, 0],
    [0, 1, 0],
    [1, 0, 0],
]).reshape([filter_num, input_channels, 3, 3])
b1 = np.zeros(filter_num)

print("weight = filter = kernel = mask is\n", W1, end='\n')

conv1 = Convolution(W1, b1)
y = conv1.forward(x1)

print("Result of Convolution ", y)



input data is 
 [[[[0 0 0 0 0]
   [0 1 2 3 0]
   [0 4 5 6 0]
   [0 7 8 9 0]
   [0 0 0 0 0]]]]
weight = filter = kernel = mask is
 [[[[0 0 0]
   [0 1 0]
   [1 0 0]]]]
input data -> im2col is  [[0. 0. 0. 0. 1. 2. 0. 4. 5.]
 [0. 0. 0. 1. 2. 3. 4. 5. 6.]
 [0. 0. 0. 2. 3. 0. 5. 6. 0.]
 [0. 1. 2. 0. 4. 5. 0. 7. 8.]
 [1. 2. 3. 4. 5. 6. 7. 8. 9.]
 [2. 3. 0. 5. 6. 0. 8. 9. 0.]
 [0. 4. 5. 0. 7. 8. 0. 0. 0.]
 [4. 5. 6. 7. 8. 9. 0. 0. 0.]
 [5. 6. 0. 8. 9. 0. 0. 0. 0.]]
Weight = filter ... -> im2col is 
 [[0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]]
affine 연상 수행 결과 
 [[ 1.]
 [ 6.]
 [ 8.]
 [ 4.]
 [12.]
 [14.]
 [ 7.]
 [ 8.]
 [ 9.]]
Result of Convolution  [[[[ 1.  6.  8.]
   [ 4. 12. 14.]
   [ 7.  8.  9.]]]]
